<a href="https://colab.research.google.com/github/DSIMB/PoincareMSA/blob/master/PoincareMSA_colab_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/DSIMB/PoincareMSA/blob/master/.github/PoincareMSA_small_logo.png?raw=true" height="100" style="height:100px;margin-left: 0px;">

# Poincaré maps for visualization of large protein famillies

**Authors**: Anna Klimovskaia Susmelj <a href="https://orcid.org/0000-0002-1057-6690" target="_blank"><img alt="ORCID logo" src="https://info.orcid.org/wp-content/uploads/2019/11/orcid_16x16.png"></a>, Yani Ren, Yann Vander Meersche <a href="https://orcid.org/0000-0002-6680-8104" target="_blank"><img alt="ORCID logo" src="https://info.orcid.org/wp-content/uploads/2019/11/orcid_16x16.png"></a>, Jean-Christophe Gelly <a href="https://orcid.org/0000-0001-5138-361X" target="_blank"><img alt="ORCID logo" src="https://info.orcid.org/wp-content/uploads/2019/11/orcid_16x16.png"></a> and Tatiana Galochkina <a href="https://orcid.org/0000-0002-3608-5208" target="_blank"><img alt="ORCID logo" src="https://info.orcid.org/wp-content/uploads/2019/11/orcid_16x16.png"></a>

<p>
	PoincaréMSA is a computational tool for visualization of large protein families developed by <a href="https://doi.org/10.1093/bib/bbad103" target="_blank" data-bs-toggle="tooltip" style="text-decoration:none;" data-bs-placement="top" title="Susmelj A.K., Ren Y., Vander Meersche Y., Gelly J.C, Galochkina T. (2023). Poincaré maps for visualization of large protein families. Briefings in bioinformatics, -. https://doi.org/10.1093/bib/bbad103">Susmelj et al.</a> PoincaréMSA builds an interactive projection of an input protein multiple sequence alignment provided by the user or built on-the-flow from the target sequence. The underlying algorithm is based on Poincaré maps introduced by <a href="https://doi.org/10.1038/s41467-020-16822-4" target="_blank" data-bs-toggle="tooltip" style="text-decoration:none;" data-bs-placement="top" title="Klimovskaia, A., Lopez-Paz, D., Bottou, L. et al. Poincaré maps for analyzing complex hierarchies in single-cell data. Nat Commun 11, 2966 (2020).">Klimovskaia et al.</a>. It successfully reproduces both local proximities between protein sequences as well as global hierarchical structure of the data.
</p>
<p>
	You can find a step-by-step explication of PoincaréMSA construction on the <a href="https://www.dsimb.inserm.fr/POINCARE_MSA/tutorial.html" target="_blank">Tutorial</a> page and several examples of PoincaréMSA projection for protein families on the <a href="https://www.dsimb.inserm.fr/POINCARE_MSA/examples.html" target="_blank">Examples</a> page. The source code is available at <a href="https://github.com/DSIMB/PoincareMSA" target="_blank">https://github.com/DSIMB/PoincareMSA</a>.<br>
</p>
<p class="mb-1"><strong>Please use the following reference when citing the PoincaréMSA:</strong></p>
<p>Susmelj A.K., Ren Y., Vander Meersche Y., Gelly J.C., &amp; Galochkina T. (2023). Poincaré maps for visualization of large protein families. <i>Briefings in bioinformatics</i>, bbad103, <a href="https://doi.org/10.1093/bib/bbad103" target="_blank">https://doi.org/10.1093/bib/bbad103</a></p>

# Notebook initialization

In [ ]:
#@title ### Load PoincaréMSA Github repository & install dependencies
print("1. Load PoincaréMSA Github repository")
import os
if os.getcwd() == "/content":
    !git clone https://github.com/DSIMB/PoincareMSA.git
    %cd PoincareMSA
    !git lfs pull

# Check if the GPU is activated
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('\nUsing device:', device)

#Install missing module
print("\n2. Install dependencies")
!pip install adjustText
!pip install -U kaleido

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#File import
from google.colab import files
import io

#Import visualization functions
from scripts.visualize_projection.pplots_new import read_embeddings, plot_embedding, plot_embedding_interactive, rotate, get_colors
from scripts.build_poincare_map.poincare_maps_rotation import *
%matplotlib inline

In [ ]:
#@title # Data selection
dataset = "thioredoxins" #@param ["kinases", "globins", "thioredoxins"]

#Select annotations
if dataset == "kinases":
    mfasta = "examples/kinases/kinases.mfasta"
    path_annotation = "examples/kinases/kinase_group_new.csv"
elif dataset == "globins":
    mfasta = "examples/globins/glob.mfasta"
    path_annotation = "examples/globins/globin_colors_new.csv"
elif dataset == "thioredoxins":
    mfasta = "examples/thioredoxins/thioredoxins.mfasta"
    path_annotation = "examples/thioredoxins/thioredoxin_annotation.csv"

#Read mfasta file
nb_seq = 0
with open(mfasta, "r") as f_in:
    for line in f_in:
        if line[0] == ">":
            nb_seq += 1
print(f"\nNumber of sequences found: {nb_seq}.")

df_annotation = pd.read_csv(path_annotation)
print("\nAnnotation file correctly loaded.")
annotation_names = ["proteins_id"] + list(df_annotation.loc[:, df_annotation.columns != 'proteins_id'].columns)
print(f"{len(annotation_names)} annotations found: {annotation_names}.")

# Data preparation
Here we clean the input .mfasta alignment and translate each sequence to a vector ready for projection.

In [ ]:
#@title ## Parameters for data preparation
#@markdown ### Job name
#@markdown Name for the output folder
out_name = "poincareMSA" #@param {type:"string"}

#@markdown ### Threshold for filtering gapped positions
#@markdown Positions with proportion of gaps above the given threshold are removed from the alignment. If your alignment is very gapped, you may want to increase this value.
gapth = 0.9 #@param {type:"number"}

#@markdown ## Run data preparation
#@markdown Data preparation consists in `.mfasta` cleaning according to a gap threshold and translation of each sequence to the PSSM profile.
prep_parameters = "scripts/prepare_data" + " " + mfasta + " " + out_name + " " + out_name + " " + str(gapth)
bash_projection = "bash scripts/prepare_data/create_projection.sh " + prep_parameters
!{bash_projection}

# Projection

In [ ]:
#@title ### Projection parameters
#@markdown Here you control different parameters of Poincaré maps. In our computational experiments the best results were achieved for the following values provided by default. The impact of different parameters is analyzed in the original paper [1].
knn = 5 #@param {type:"number"}
gamma = 2 #@param {type:"number"}
sigma = 1 #@param {type:"number"}
cospca = 0 #@param {type:"number"}
batchs = 4 #@param {type:"number"}
epochs = 1000 #@param {type:"number"}
seed = 0 #@param {type:"number"}

#@markdown ## Building projection and preparing data for visualization
#@markdown This step creates a projection of encoded sequences to a Poincaré disk.
bash_pm = "python3 "+ "scripts/build_poincare_map/main.py --input_path " + out_name + "/fasta" + str(gapth) + " --output_path " + out_name + "/projections/ --gamma "+ str(gamma) +" --pca "+ str(cospca) + " --epochs "+ str(epochs) +" --seed "+ str(seed) + " --knn " + str(knn)
!{bash_pm}

# Projection visualization

In [ ]:
#@title ### Prepare data for visualization
#@markdown Check if the annotation file is provided and prepares a dataframe for visualization.

#Check that an annotation file was provided. Create a dummy one instead
if not path_annotation:
    df_annotation = pd.DataFrame(list(range(1,nb_seq+1)), columns=["proteins_id"], dtype="float")
    df_annotation.to_csv("dummy_annotation.csv", index=False)
    path_annotation = "dummy_annotation.csv"
    annotation_names = ["proteins_id"]

#Create the DataFrame for visualization
path_embedding = f"{out_name}/projections/PM{knn:1.0f}sigma={sigma:2.2f}gamma={gamma:2.2f}cosinepca={cospca:1.0f}_seed{seed:1.0f}.csv"
df_embedding = read_embeddings(path_embedding, path_annotation, withroot=False)

# save original coordinate before any rotation
df_embedding["pm1_com"] = df_embedding["pm1"]
df_embedding["pm2_com"] = df_embedding["pm2"]

#@markdown Here are different labels found in your annotation file (if one uploaded):
print(f"{len(annotation_names)} annotations found: {annotation_names}.")

In [ ]:
#@title ### Create interactive plot
#@markdown Here you can set different parameters to color & annotate the resulting projection.
#@markdown One can also provide a protein ID used to center your projection.


title = "" #@param {type:"string"}

#Labels name
#@markdown ---
#@markdown #### Select the coloring from annotation .csv file:
labels_name = "" #@param {type:"string"}
if labels_name == "":
    labels_name = None
elif labels_name not in annotation_names:
    raise NameError(f"labels_name {labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}")

#Labels text
#@markdown #### Select classes to label among the "labels_name" or "second_labels_name" column (comma separated list):
second_labels_name = "" #@param {type:"string"}
if second_labels_name == "":
    second_labels_name = None
elif second_labels_name not in annotation_names:
    raise NameError(f'"second_labels_name" {second_labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}')

labels_text = "" #@param {type:"string"}
if labels_text:
    try:
        labels_text = [s.strip() for s in labels_text.split(",")]
    except:
        print('Error: "label_text" field is not a valid list.')
else:
    labels_text = [""]

#Convert labels_text to labels_name dtype
if labels_name and second_labels_name is None:
    if labels_name and labels_text != [""]:
        try:
            labels_text_dtype = df_annotation[labels_name].dtypes
            labels_text = list(np.array(labels_text).astype(labels_text_dtype))
        except:
            raise TypeError(f'"labels_text" is not compatible with {labels_name}" data format ({labels_text_dtype}).')
else:
    if second_labels_name and labels_text != [""]:
        try:
            labels_text_dtype = df_annotation[second_labels_name].dtypes
            labels_text = list(np.array(labels_text).astype(labels_text_dtype))
        except:
            raise TypeError(f'"labels_text" is not compatible with {second_labels_name}" data format ({labels_text_dtype}).')

show_text = True #@param {type:"boolean"}
#@markdown ---

#@markdown #### Use a custom color palette:
color_palette = None #@param {type:"raw"}
use_custom_palette = False #@param {type:"boolean"}

if not use_custom_palette:
    color_palette = None

#@markdown ---

#@markdown #### Select projection center:

id_for_rotation = None #@param {type:"number"}
use_selected_center = False #@param {type:"boolean"}

tmp_id_for_rotation = -1
if use_selected_center and id_for_rotation is not None:
    if id_for_rotation != tmp_id_for_rotation:
        tmp_id_for_rotation = id_for_rotation

        poincare_coord = df_embedding[["pm1","pm2"]].sort_values(by=["proteins_id"])
        poincare_coord = poincare_coord.values[:,:2]
        model = PoincareMaps(poincare_coord)
        model.get_distances()

        model.iroot = id_for_rotation - 1  #Zero indexing
        model.rotate()

        df_embedding["pm1"] = model.coordinates_rotated[:,0]
        df_embedding["pm2"] = model.coordinates_rotated[:,1]
else:
    df_embedding["pm1"] = df_embedding["pm1_com"]
    df_embedding["pm2"] = df_embedding["pm2_com"]

#Plot graph
fig = plot_embedding_interactive(df_embedding, 
                                 labels_name = labels_name,
                                 second_labels_name = second_labels_name, 
                                 show_text = show_text,
                                 labels_text = labels_text,
                                 color_palette = color_palette, 
                                 title = title, 
                                 fontsize = 11)
fig.show()

In [ ]:
#@title Save plot to file
output_name = "figure" #@param {type:"string"}
output_format = "html" #@param ["png", "html", "pdf", "svg"]

if output_format != "html":
    fig.write_image(f"{output_name}.{output_format}", engine="kaleido")
else:
    fig.write_html(f"{output_name}.{output_format}")
files.download(f"{output_name}.{output_format}")

In [ ]:
#@title Download intermediate data
bash_command = f"zip -r -q {out_name}.zip {out_name}"
!{bash_command}

files.download(f"{out_name}.zip")

# Help

### Enabling the GPU

To enable GPU in your notebook, select the following menu options −
```
Runtime / Change runtime type
```

<figure>
<center>
<img src="https://github.com/DSIMB/PoincareMSA/blob/master/.github/colab_gpu.png?raw=true" width=500>
</center>
</figure>

